In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

import m2cgen as m2c

In [24]:
data = np.genfromtxt("./build/data/gamestate0.csv", skip_header=1, delimiter=",")
y = data[:, 0]
X = data[:, 1:]
X = np.nan_to_num(X, nan=0.0)

test_split = 500
# 2. Split into training and test sets
X_train, X_test, y_train, y_test = X[test_split:, :], X[:test_split, :],  y[test_split:], y[:test_split]


# data_train = np.genfromtxt("./build/data/gamestate0_train_4rm.csv", skip_header=1, delimiter=",")
# data_test = np.genfromtxt("./build/data/gamestate0_test_4rm.csv", skip_header=1, delimiter=",")

# y_train = data_train[:, 0]
# X_train = data_train[:, 1:]
# X_train = np.nan_to_num(X_train, nan=0.0)

# y_test = data_test[:, 0]
# X_test = data_test[:, 1:]
# X_test = np.nan_to_num(X_test, nan=0.0)

# print(data.shape)

In [25]:

# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a basic classifier (Random Forest here)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

# 5. Make predictions
y_pred = clf.predict(X_test_scaled)

# 6. Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.794

Classification Report:
               precision    recall  f1-score   support

        -1.0       0.80      0.89      0.84       235
         0.0       0.00      0.00      0.00        46
         1.0       0.79      0.86      0.83       219

    accuracy                           0.79       500
   macro avg       0.53      0.58      0.55       500
weighted avg       0.72      0.79      0.76       500



/home/mason/cs4701_sp25/second_attempt/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mason/cs4701_sp25/second_attempt/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mason/cs4701_sp25/second_attempt/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [27]:
code = m2c.export_to_c(clf)

with open("src/model.c", "w") as f:
    f.write(code)

# DONT FORGET THE SCALARS

In [29]:
scales = ", ".join(str(x) for x in list(scaler.scale_))
means = ", ".join(str(x) for x in list(scaler.mean_))

params = "constexpr double means[] = {" + means + "};\nconstexpr double scale[] = {" + scales + "};"
print(params)

constexpr double means[] = {6.83847066578779, 0.5862443935398807, 6.029848384970357, 0.5347017079762681, 85.96901779828609, 66.71061305207647, 34.27686222808174, 29.75148319050758};
constexpr double scale[] = {4.145915610629083, 0.22465748132417496, 4.313175264272859, 0.26418571683119374, 87.05417184576838, 78.84886187221142, 15.585342865572235, 17.40184468742501};


[11.772048524951849,
 0.937536590432978,
 10.75941549489949,
 0.8741966229667335,
 148.55913978494624,
 123.75241246208988,
 36.92197408326441,
 34.059001929969675]